In [4]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
from scipy.integrate import odeint

In [2]:
##### CONSTANTS and DEFINATIONS #######
us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'American Samoa': 'AS',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Guam': 'GU',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY'
}
county_dict = {"Kings County, New York":"Kings",
    "Clark County, Nevada":"Clark",
    "San Bernardino County, California":"San Bernardino",
    "Wayne County, Michigan":"Wayne",
    "Tarrant County, Texas":"Tarrant",
    "Franklin County, Ohio":"Franklin",
    "Cuyahoga County, Ohio":"Cuyahoga",
    "Miami-Dade County, Florida":"Miami-Dade",
    "Bexar County, Texas":"Bexar",
    "Maricopa County, Arizona":"Maricopa",
    "Cook County, Illinois":"Cook",
    "Los Angeles County, California":"Los Angeles",
    "Philadelphia County, Pennsylvania":"Philadelphia",
    "San Diego County, California":"San Diego",
    "Suffolk County, Massachusetts":"Suffolk",
    "Broward County, Florida":"Broward",
    "Hennepin County, Minnesota":"Hennepin",
    "Allegheny County, Pennsylvania":"Allegheny",
    "Alameda County, California":"Alameda",
    "Orange County, California":"Orange"}


In [220]:
def create_SIR_model(county_dict):
    
    population_df = pd.read_csv('covid_county_population_usafacts.csv')
    covid_data = pd.read_csv('covid_cases.csv', index_col=0)
    covid_data.index = pd.to_datetime(covid_data.index)
    data = pd.DataFrame()
    county_idx = []
    infection_1y = []
    infection_2y = []
    infection_5y = []
    infected_prop = []
    infected_t = []
    
    ## Data Wrangling
    for key in county_dict:
        county, state = (key.replace(' County','').strip('')).split(',')
        state = state.strip()
        county = county.strip()
        label = county + ' ' + state
        
        N = population_df.loc[(population_df['County Name'] == f'{county} County' ) & (population_df.State == f'{us_state_abbrev[state]}')].values[0][3]
        
        print(county, state, N)
        
        df_county = covid_data[[label + " Cases", label +" Deaths"]]
        
        condition = df_county.values[:,1] > 0
        df_county = df_county.iloc[condition,:]
        
        training_size = len(df_county)
        
        infected = df_county[label +" Cases"].values
        recovered = df_county[label +" Deaths"].values
        susceptible = N - infected- recovered
        
        # Initial number of infected and recovered individuals, I0 and R0.
        I0, R0 = infected[0]+1, recovered[0]
        # Everyone else, S0, is susceptible to infection initially.
        S0 = N - I0 - R0
        # Initial conditions vector
        y0 = S0, I0, R0



        #Curve fit SIR model to canadian data
        optimal = minimize(SIR_MSE,
                           [0.0001, 0.0001],
                           args=(susceptible, infected,recovered, S0, I0, R0, N),
                           bounds=[(0, 1), (0, 1)],
                           method = 'Nelder-Mead'
                           )
        beta, gamma = optimal.x
        print("---------------")
        print("SIR Model - Optimal Parameters:")
        print("Beta: ",beta)
        print("Gamma: ",gamma)

        date_range = pd.date_range(start=df_county.iloc[0].name, periods=training_size + 5*365, freq='1D')

        DAYS = training_size + 365*5#len(infected)
        t = np.linspace(0, DAYS-1, DAYS)

        solution = odeint(SIR, y0, t, args=(N,beta, gamma))
        S, I, R = solution.T

        print(len(date_range))
        print(len(I))
        print(f"Infection number in one year for {label}")
        print(I[training_size+365-1])
        print(f"Infection number in two years for {label}")
        print(I[training_size+2*365-1])
        print(f"Infection number in five years for {label}")
        print(I[training_size+5*365-1])
        print('---------------------------------')
        
        county_idx.append(label)
        infection_1y.append(I[training_size+365-1])
        infection_2y.append(I[training_size+2*365-1])
        infection_5y.append(I[training_size+5*365-1])
        infected_prop.append(infected[-1]/N)
        infected_t.append(infected[-1])
        
        temp = pd.DataFrame(index = [label], columns = date_range, data = [I])
        
        data = pd.concat([data, temp], axis = 0, sort=False)
            
        
    data_dict = {
        'County': county_idx,
        '1Y Prediction' : infection_1y,
        '2Y Prediction' : infection_2y,
        '5Y Prediction' : infection_5y,
        'Infected Proportion': infected_prop,
        'Last # of Infected' : infected_t
    }
    
    data_frame = pd.DataFrame.from_dict(data_dict)
    data_frame.set_index('County', inplace= True)
    
    data.fillna(0, inplace=True)
    
    data_frame.to_csv('SIR_summary.csv')
    data.to_csv('Predicted time series per county.csv')
    
    return data_frame, data
        

#         plt.plot(S)
#         plt.plot(I)
#         plt.plot(R)



In [217]:
#### Models
def SIR(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt
#Compute Mean-Squared Error for SIR
def SIR_MSE( point, susceptible, infected, recovered, s_0, i_0, r_0, N):
    timeSteps = np.linspace(0, len(infected), len(infected))
    beta, gamma = point
    # Initial conditions vector
    y0 = s_0, i_0, r_0
    # Integrate the SIR equations over the time grid, t.
    solution = odeint(SIR, y0, timeSteps, args=(N, beta, gamma))
    S, I, R = solution.T
    S_loss = np.sqrt(np.mean((S - susceptible) ** 2))
    I_loss = np.sqrt(np.mean((I - infected) ** 2))
    R_loss = np.sqrt(np.mean((R - recovered) ** 2))
    return (S_loss + I_loss + R_loss)/3

In [219]:
test_V2 = create_SIR_model(county_dict)

Kings New York 2559903
---------------
SIR Model - Optimal Parameters:
Beta:  0.03910333513566906
Gamma:  0.004096031812006429
2056
2056
Infection number in one year for Kings New York
834292.300411087
Infection number in two years for Kings New York
188330.54591740342
Infection number in five years for Kings New York
2131.187076349722
---------------------------------
Clark Nevada 2266715
---------------
SIR Model - Optimal Parameters:
Beta:  0.036767434475970806
Gamma:  0.0007565137993620125
2054
2054
Infection number in one year for Clark Nevada
1826728.226988062
Infection number in two years for Clark Nevada
1386110.0028683913
Infection number in five years for Clark Nevada
605390.6455963058
---------------------------------
San Bernardino California 2180085
---------------
SIR Model - Optimal Parameters:
Beta:  0.03672378703661772
Gamma:  0.0006489541202006442
2045
2045
Infection number in one year for San Bernardino California
1817524.3782047906
Infection number in two years for 

In [213]:
test_V2

,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,...,2025-10-20,2025-10-21,2025-10-22,2025-10-23,2025-10-24,2025-10-25,2025-10-26,2025-10-27,2025-10-28,2025-10-29
Kings New York,NaN,NaN,NaN,42.000000,43.496316,45.045940,46.650770,48.312774,50.033987,51.816520,...,2.211162e+03,2.202129e+03,2.193134e+03,2.184176e+03,2.175253e+03,2.166368e+03,2.157519e+03,2.148705e+03,2.139928e+03,2.131187e+03
Clark Nevada,NaN,NaN,NaN,NaN,NaN,36.000000,37.319995,38.688389,40.106956,41.577536,...,6.095266e+05,6.090656e+05,6.086051e+05,6.081448e+05,6.076849e+05,6.072254e+05,6.067662e+05,6.063073e+05,6.058488e+05,6.053906e+05
San Bernardino California,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,7.088890e+05,7.084291e+05,7.079695e+05,7.075103e+05,7.070513e+05,7.065926e+05,7.061342e+05,7.056761e+05,7.052183e+05,7.047608e+05
Wayne Michigan,NaN,2.000000,2.092140,2.188526,2.289352,2.394823,2.505153,2.620566,2.741295,2.867588,...,1.700876e+03,1.694109e+03,1.687370e+03,1.680657e+03,1.673971e+03,1.667312e+03,1.660679e+03,1.654072e+03,1.647492e+03,1.640938e+03
Tarrant Texas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.000000,9.380244,9.776554,...,7.463392e+05,7.458958e+05,7.454526e+05,7.450097e+05,7.445671e+05,7.441247e+05,7.436826e+05,7.432407e+05,7.427991e+05,7.423578e+05
Franklin Ohio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.488657e+05,3.485953e+05,3.483251e+05,3.480552e+05,3.477854e+05,3.475159e+05,3.472465e+05,3.469774e+05,3.467085e+05,3.464398e+05
Cuyahoga Ohio,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.318333e+05,2.316005e+05,2.313680e+05,2.311357e+05,2.309036e+05,2.306718e+05,2.304402e+05,2.302088e+05,2.299777e+05,2.297468e+05
Miami-Dade Florida,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.037007e+06,1.036433e+06,1.035859e+06,1.035286e+06,1.034713e+06,1.034140e+06,1.033567e+06,1.032995e+06,1.032423e+06,1.031851e+06
Bexar Texas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.780006e+05,4.776051e+05,4.772099e+05,4.768150e+05,4.764205e+05,4.760263e+05,4.756324e+05,4.752388e+05,4.748456e+05,4.744527e+05
Maricopa Arizona,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,8.453242e+05,8.445150e+05,8.437066e+05,8.428990e+05,8.420921e+05,8.412860e+05,8.404807e+05,8.396761e+05,8.388724e+05,8.380693e+05


In [221]:
def clean_jhu_data(county_dict):
    
    ## Reading in Data from JHU Github
    us_cases = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")
    us_death = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv")
    
    ##
    data = pd.DataFrame()
    
    ## Data Wrangling
    for key in county_dict:
        county, state = (key.replace(' County','').strip('')).split(',')
        state = state.strip()
        county = county.strip()
     
        
        temp_cases_df = us_cases.loc[(us_cases.Admin2 == county) & (us_cases.Province_State == state)].T
        temp_cases_df = temp_cases_df[11:]
        temp_cases_df.columns = [f'{county} {state} Cases']
        temp_cases_df.index = pd.to_datetime(temp_cases_df.index)
        
        temp_death_df = us_death.loc[(us_death.Admin2 == county) & (us_death.Province_State == state)].T
        temp_death_df = temp_death_df[12:]
        temp_death_df.columns = [f'{county} {state} Deaths']
        temp_death_df.index = pd.to_datetime(temp_death_df.index)
        
        data = pd.concat([data, temp_cases_df, temp_death_df], axis =1 , sort=False)
    
    return data